# 参数初始化
参数初始化对模型具有较大的影响，不同的初始化方式可能会导致截然不同的结果，所幸的是很多深度学习的先驱们已经帮我们探索了各种各样的初始化方式，所以我们只需要学会如何对模型的参数进行初始化的赋值即可。

PyTorch 的初始化方式并没有那么显然，如果你使用最原始的方式创建模型，那么你需要定义模型中的所有参数，当然这样你可以非常方便地定义每个变量的初始化方式，但是对于复杂的模型，这并不容易，而且我们推崇使用 Sequential 和 Module 来定义模型，所以这个时候我们就需要知道如何来自定义初始化方式

## 使用 NumPy 来初始化
因为 PyTorch 是一个非常灵活的框架，理论上能够对所有的 Tensor 进行操作，所以我们能够通过定义新的 Tensor 来初始化，直接看下面的例子

In [6]:
import numpy as np
import torch
from torch import nn

In [13]:
# 定义一个 Sequential 模型
net1 = nn.Sequential(
    nn.Linear(30, 40),
    nn.ReLU(),
    nn.Linear(40, 50),
    nn.ReLU(),
    nn.Linear(50, 10)
)

In [14]:
# 访问第一层的参数
w1 = net1[0].weight
b1 = net1[0].bias

In [15]:
print(w1)

Parameter containing:
-0.0221 -0.0171  0.1221  ...  -0.0452 -0.1715 -0.0637
-0.0922 -0.1111  0.0822  ...   0.0316  0.1020 -0.0585
-0.0830  0.1037 -0.0572  ...  -0.1465 -0.1049 -0.0566
          ...             ⋱             ...          
 0.1485  0.1137  0.1745  ...   0.0073  0.0887  0.1143
 0.1634 -0.1478  0.0930  ...   0.1418 -0.0501  0.1266
 0.0943 -0.1595 -0.1742  ...  -0.1531  0.0786 -0.1594
[torch.FloatTensor of size 40x30]



注意，这是一个 Parameter，也就是一个特殊的 Variable，我们可以访问其 `.data`属性得到其中的数据，然后直接定义一个新的 Tensor 对其进行替换，我们可以使用 PyTorch 中的一些随机数据生成的方式，比如 `torch.randn`，如果要使用更多 PyTorch 中没有的随机化方式，可以使用 numpy

In [16]:
# 定义一个 Tensor 直接对其进行替换
net1[0].weight.data = torch.from_numpy(np.random.uniform(3, 5, size=(40, 30)))

In [17]:
print(net1[0].weight)

Parameter containing:
 4.6904  3.5478  4.0254  ...   3.6078  4.6897  4.8285
 4.6349  3.4475  4.8485  ...   3.8712  3.9396  4.7797
 3.0177  3.4870  4.5741  ...   4.6718  4.2548  4.6343
          ...             ⋱             ...          
 3.3116  3.2907  4.5550  ...   3.5882  4.4668  3.6532
 4.2998  4.6337  3.8836  ...   3.1220  4.0567  4.3605
 4.3862  4.5433  4.1909  ...   4.2792  4.7513  3.7076
[torch.DoubleTensor of size 40x30]



可以看到这个参数的值已经被改变了，也就是说已经被定义成了我们需要的初始化方式，如果模型中某一层需要我们手动去修改，那么我们可以直接用这种方式去访问，但是更多的时候是模型中相同类型的层都需要初始化成相同的方式，这个时候一种更高效的方式是使用循环去访问，比如

In [18]:
for layer in net1:
    if isinstance(layer, nn.Linear): # 判断是否是线性层
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 
        # 定义为均值为 0，方差为 0.5 的正态分布

**小练习：一种非常流行的初始化方式叫 Xavier，方法来源于 2010 年的一篇论文 [Understanding the difficulty of training deep feedforward neural networks](http://proceedings.mlr.press/v9/glorot10a.html)，其通过数学的推到，证明了这种初始化方式可以使得每一层的输出方差是尽可能相等的，有兴趣的同学可以去看看论文**

我们给出这种初始化的公式

$$
w\ \sim \ Uniform[- \frac{\sqrt{6}}{\sqrt{n_j + n_{j+1}}}, \frac{\sqrt{6}}{\sqrt{n_j + n_{j+1}}}]
$$

其中 $n_j$ 和 $n_{j+1}$ 表示该层的输入和输出数目，所以请尝试实现以下这种初始化方式

对于 Module 的参数初始化，其实也非常简单，如果想对其中的某层进行初始化，可以直接像 Sequential 一样对其 Tensor 进行重新定义，其唯一不同的地方在于，如果要用循环的方式访问，需要介绍两个属性，children 和 modules，下面我们举例来说明

In [28]:
class sim_net(nn.Module):
    def __init__(self):
        super(sim_net, self).__init__()
        self.l1 = nn.Sequential(
            nn.Linear(30, 40),
            nn.ReLU()
        )
        
        self.l1[0].weight.data = torch.randn(30, 40) # 直接对某一层初始化
        
        self.l2 = nn.Sequential(
            nn.Linear(40, 50),
            nn.ReLU()
        )
        
        self.l3 = nn.Sequential(
            nn.Linear(50, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.l1(x)
        x =self.l2(x)
        x = self.l3(x)
        return x

In [29]:
net2 = sim_net()

In [30]:
# 访问 children
for i in net2.children():
    print(i)

Sequential(
  (0): Linear(in_features=30, out_features=40)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=40, out_features=50)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=50, out_features=10)
  (1): ReLU()
)


In [31]:
# 访问 modules
for i in net2.modules():
    print(i)

sim_net(
  (l1): Sequential(
    (0): Linear(in_features=30, out_features=40)
    (1): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=40, out_features=50)
    (1): ReLU()
  )
  (l3): Sequential(
    (0): Linear(in_features=50, out_features=10)
    (1): ReLU()
  )
)
Sequential(
  (0): Linear(in_features=30, out_features=40)
  (1): ReLU()
)
Linear(in_features=30, out_features=40)
ReLU()
Sequential(
  (0): Linear(in_features=40, out_features=50)
  (1): ReLU()
)
Linear(in_features=40, out_features=50)
ReLU()
Sequential(
  (0): Linear(in_features=50, out_features=10)
  (1): ReLU()
)
Linear(in_features=50, out_features=10)
ReLU()


通过上面的例子，看到区别了吗?

children 只会访问到模型定义中的第一层，因为上面的模型中定义了三个 Sequential，所以只会访问到三个 Sequential，而 modules 会访问到最后的结构，比如上面的例子，modules 不仅访问到了 Sequential，也访问到了 Sequential 里面，这就对我们做初始化非常方便，比如

In [33]:
for layer in net2.modules():
    if isinstance(layer, nn.Linear):
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 

这上面实现了和 Sequential 相同的初始化，同样非常简便

## torch.nn.init
因为 PyTorch 灵活的特性，我们可以直接对 Tensor 进行操作从而初始化，PyTorch 也提供了初始化的函数帮助我们快速初始化，就是 `torch.nn.init`，其操作层面仍然在 Tensor 上，下面我们举例说明

In [34]:
from torch.nn import init

In [38]:
print(net1[0].weight)

Parameter containing:
 0.2051 -0.4551  0.7049  ...   0.5223 -0.7658 -0.1899
 0.2562  0.2797  0.0012  ...  -0.5278 -0.4887 -0.8263
 0.4582 -0.1433  0.5009  ...   0.1000 -0.5663  0.1605
          ...             ⋱             ...          
 0.8715  0.4053  0.3679  ...  -0.4733 -0.6270 -0.3325
-0.1898  0.6608  0.1111  ...   0.2294  0.2603 -0.0200
-0.3035  0.1876 -0.5422  ...   0.0505  0.6244 -0.2368
[torch.DoubleTensor of size 40x30]



In [39]:
init.xavier_uniform(net1[0].weight) # 这就是上面我们讲过的 Xavier 初始化方法，PyTorch 直接内置了其实现

Parameter containing:
-0.0449 -0.2140  0.2820  ...  -0.2266  0.0365 -0.1897
-0.0313  0.1128  0.1789  ...  -0.1731  0.0590 -0.1085
-0.0347 -0.1429 -0.1646  ...   0.0212  0.1731 -0.0251
          ...             ⋱             ...          
 0.0902 -0.1555  0.0562  ...  -0.0109 -0.2192 -0.1540
-0.1491 -0.2610 -0.2453  ...   0.2201  0.2257  0.1047
 0.0297  0.1414 -0.0139  ...  -0.1209 -0.0193 -0.1731
[torch.DoubleTensor of size 40x30]

In [40]:
print(net1[0].weight)

Parameter containing:
-0.0449 -0.2140  0.2820  ...  -0.2266  0.0365 -0.1897
-0.0313  0.1128  0.1789  ...  -0.1731  0.0590 -0.1085
-0.0347 -0.1429 -0.1646  ...   0.0212  0.1731 -0.0251
          ...             ⋱             ...          
 0.0902 -0.1555  0.0562  ...  -0.0109 -0.2192 -0.1540
-0.1491 -0.2610 -0.2453  ...   0.2201  0.2257  0.1047
 0.0297  0.1414 -0.0139  ...  -0.1209 -0.0193 -0.1731
[torch.DoubleTensor of size 40x30]



可以看到参数已经被修改了

`torch.nn.init` 为我们提供了更多的内置初始化方式，避免了我们重复去实现一些相同的操作

上面讲了两种初始化方式，其实它们的本质都是一样的，就是去修改某一层参数的实际值，而 `torch.nn.init` 提供了更多成熟的深度学习相关的初始化方式，非常方便

下一节课，我们将讲一下目前流行的各种基于梯度的优化算法